# EXCERCISE 1 - COLLEGE

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession

## Nhập dữ liệu

In [2]:
sc= SparkContext(master= 'local', appName= 'Chapter 7 - Exercise1')
ss= SparkSession(sc)

In [3]:
path= '/Users/vovanthuong/Desktop/9 - Big Data in Machine Learning/Data/Chapter6/College.csv'
df= ss.read.csv(path, header= True, inferSchema= True)

## Kiểm tra dữ liệu

In [4]:
df.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



- School: Tên trường: Loại bỏ
- Private: Tư nhân -> biến outcomt. Mã hóa Yes/No -> 1/0
- Apps: Số lượng đơn ứng tuyển
- Accept: Số lượng trúng tuyển
- Enroll: Số lượng sinh viên đến ghi danh
- Top10perc: tỷ lệ sinh viên đến từ top 10% của trường THPT
- Top20perc: tỷ lệ sinh viên đến từ top 20% của trường THPT
- F_Undergrad: Số lượng sinh viên đại học toàn thời gian
- P_Undergrad: Số lượng sinh viên đại học bán thời gian
- Outstate: Học phí ngoài tiểu bang
- Room_Board: Chi phí sinh hoạt chung
- Books: Ước lượng chi phí sách
- Personal: Ước lượng chi tiêu cá nhân
- PhD: Tỷ lệ giảng viên là PhD
- Terminal: Tỷ lệ giảng viên là terminal degree (bằng sau đại học nhưng không phải Master, PhD)
- S_F_Ratio: Tỷ lệ sinh viên/giảng viên
- perc_alumni: Tỷ lệ cự sinh viên quyên góp
- Expend: Chi phí giảng giải trên mỗi sinh viên
- Grad_Rate: Tỷ lệ tốt nghiệp

In [22]:
from pyspark.sql.functions import count, when, isnan, isnull, col
nan_data= df.select([count(when(isnan(c), c)).alias(c + '_nan') for c in df.columns]).toPandas().T
nan_data

,0
School_nan,0
Private_nan,0
Apps_nan,0
Accept_nan,0
Enroll_nan,0
Top10perc_nan,0
Top25perc_nan,0
F_Undergrad_nan,0
P_Undergrad_nan,0
Outstate_nan,0


In [23]:
null_data= df.select([count(when(isnull(c), c)).alias(c + '_null') for c in df.columns]).toPandas().T
null_data

,0
School_null,0
Private_null,0
Apps_null,0
Accept_null,0
Enroll_null,0
Top10perc_null,0
Top25perc_null,0
F_Undergrad_null,0
P_Undergrad_null,0
Outstate_null,0


## Tách tập train và test

In [17]:
train, test= df.randomSplit([0.8, 0.2])

## Tạo Pipe xử lý dữ liệu

In [18]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline

In [16]:
private_idxer= StringIndexer(inputCol= 'Private', outputCol= 'label')

In [20]:
input_cols= ['Apps', 'Accept', 'Enroll', 'Top10perc', 'Top25perc', 'F_Undergrad', 'P_Undergrad', 'Outstate', 'Room_Board', 'Books', 'Personal', 'PhD', 'Terminal', 'S_F_Ratio', 'perc_alumni', 'Expend', 'Grad_Rate']
vector_assembler= VectorAssembler(inputCols= input_cols, outputCol= 'features')

In [21]:
pipe_process= Pipeline(stages=[private_idxer, vector_assembler])
process_model= pipe_process.fit(train)

## Tạo mô hình

In [24]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier

In [26]:
train_cleaned= process_model.transform(train)

In [55]:
# DecisionTreeClassifier
dt_classifier= DecisionTreeClassifier(featuresCol= 'features')
dt_classifier_model= dt_classifier.fit(train_cleaned)

# RandomForestClassifier
rf_classifier= RandomForestClassifier(featuresCol= 'features', numTrees= 30)
rf_classifier_model= rf_classifier.fit(train_cleaned)

# GBTClassifier
gbt_classifier= GBTClassifier(featuresCol= 'features')
gbt_classifier_model= gbt_classifier.fit(train_cleaned)

## Đánh giá mô hình

In [61]:
def model_evaluator(data):
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
    mutil_classification_evaluator= MulticlassClassificationEvaluator(labelCol= 'label')
    accuracy= mutil_classification_evaluator.setMetricName('accuracy')
    f1= mutil_classification_evaluator.setMetricName('f1')
    bi_classification_evaluator= BinaryClassificationEvaluator(labelCol= 'label')
    auc= bi_classification_evaluator.setMetricName('areaUnderROC')
    for model in [dt_classifier_model, rf_classifier_model, gbt_classifier_model]:
        print(model)
        data_result= model.transform(data)
        data_result.groupBy('label', 'prediction').count().show()
        print('Accuracy: ', accuracy.evaluate(data_result))
        print('F1: ', f1.evaluate(data_result))
        print('AUC: ', auc.evaluate(data_result))
        print()

### Đánh giá trên tập train

In [62]:
model_evaluator(train_cleaned)

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_2066926f75a2) of depth 5 with 39 nodes
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  169|
|  0.0|       1.0|    9|
|  1.0|       0.0|    8|
|  0.0|       0.0|  452|
+-----+----------+-----+

Accuracy:  0.973377295553552
F1:  0.973377295553552
AUC:  0.9794661568440016

RandomForestClassificationModel (uid=RandomForestClassifier_f693f14388a6) with 30 trees
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  164|
|  0.0|       1.0|    2|
|  1.0|       0.0|   13|
|  0.0|       0.0|  459|
+-----+----------+-----+

Accuracy:  0.9762506264199967
F1:  0.9762506264199967
AUC:  0.9959802443717294

GBTClassificationModel (uid=GBTClassifier_1ccd8d12170a) with 20 trees
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  177|
|  0.0|       1.0|    1|
|  0.0|       0.0|  460|
+-----+----------+-----+

Accurac

### Đánh giá trên tập test

In [37]:
test_cleaned= process_model.transform(test)

In [63]:
model_evaluator(test_cleaned)

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_2066926f75a2) of depth 5 with 39 nodes
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   29|
|  0.0|       1.0|    5|
|  1.0|       0.0|    6|
|  0.0|       0.0|   99|
+-----+----------+-----+

Accuracy:  0.9204791772988932
F1:  0.9204791772988932
AUC:  0.9703296703296703

RandomForestClassificationModel (uid=RandomForestClassifier_f693f14388a6) with 30 trees
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   31|
|  0.0|       1.0|    2|
|  1.0|       0.0|    4|
|  0.0|       0.0|  102|
+-----+----------+-----+

Accuracy:  0.9564052959313221
F1:  0.9564052959313221
AUC:  0.9945054945054945

GBTClassificationModel (uid=GBTClassifier_1ccd8d12170a) with 20 trees
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   29|
|  0.0|       1.0|    4|
|  1.0|       0.0|    6|
|  0.0|       0.0|  100|
+-----